# Introduction

This tutorial shows how to classify images from fingerprints to originating from a left or right hand using a tf.keras.Sequential model .

1. Setup
1. Load the data
1. Build and split the data into Train, Validation and Test
1. Label the datasets
1. Apply Augmentation to the datasets
1. Optimize the dataset
1. Define the model structure, a training strategy, hyperparameter tuning, and compile
1. Perform a Random Search over Hyperparameters
1. Train and Hypertune model
1. Evaluate the model
1. Predict on new data
1. Save the model

In addition, the _compression notebook demonstrates how to convert a saved model to a TensorFlow Lite model for on-device machine learning on mobile, embedded, and IoT devices.

References: 
1. https://www.kaggle.com/datasets/ruizgara/socofing
1. https://www.tensorflow.org/tutorials/images/classification
1. https://pyimagesearch.com/2021/06/07/easy-hyperparameter-tuning-with-keras-tuner-and-tensorflow/
1. https://keras.io/guides/distributed_training/

# Setup

In [ ]:
# quiet install of requirements
!pip install -U pip tensorflow -q
!pip install -r ../../requirements.txt -q

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("tensorflow version:  " + tf.__version__)


# '0' Log all messages.
# '1' Log all messages except INFO.
# '2' Log all messages except INFO and WARNING. (default)
# '3' Log all messages except INFO, WARNING, and ERROR.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

In [ ]:
# this directory is apart of the .gitignore to ensure it is not committed to git
%env SCRATCH=../scratch
![ -e "${SCRATCH}" ] || mkdir -p "${SCRATCH}"/model

import os
scratch_path = os.environ.get('SCRATCH', '../scratch')

# Download data

In [ ]:
!mkdir ../scratch/{hand,model}

In [ ]:
!tar -xJf ./compressed_data/left.xz -C ../scratch/hand/

In [ ]:
!tar -xJf ./compressed_data/right.xz -C ../scratch/hand/

In [ ]:
!tar -xJf ./compressed_data/real.xz -C ../scratch/ 

# Split the data into Train, Validation and Test

Use .take() and .skip() to further split the validation_ds set into 2 datasets -- one for validation and the other for test. Let's assume that you need 80% for training set, 10% for validation set, and 10% for test set. Determine how many batches of data are available in the validation set using tf.data.experimental.cardinality, and then move the two-third of them (2/3 of 30% = 20%) to a test set as follows. Note that the default value of batch_size is 32

All the three datasets (train_ds, val_ds, and test_ds) yield batches of images together with labels inferred from the directory structure.

- Training Dataset: The sample of data used to fit the model.
- Validation Dataset: The sample of data used to provide an unbiased evaluation of a model fit on the training dataset while tuning model hyperparameters. The evaluation becomes more biased as skill on the validation dataset is incorporated into the model configuration.
- Test Dataset: The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset.# Create training datasets

In [ ]:
# define parameters for the loader

img_height = 96
img_width = 96
batch_size = 32

It's good practice to use a validation split when developing your model. We will use 80% of the images for training, and 20% for validation.

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    # Directory where the data is located. If labels is "inferred", it should contain subdirectories, each containing images for a class. 
    '../scratch/hand',
    
    # Either "inferred" (labels are generated from the directory structure), None (no labels), or a list/tuple of integer labels of the same size as the number of image files found in the directory. 
    labels='inferred',
    
    # String describing the encoding of labels. 'int': means that the labels are encoded as integers. (e.g. for sparse_categorical_crossentropy loss).
    # 'categorical' means that the labels are encoded as a categorical vector.  (e.g. for categorical_crossentropy loss).
    #'binary' means that the labels (there can be only 2) are encoded as float32 scalars with values 0 or 1. (e.g. for binary_crossentropy).
    label_mode = "categorical", 
    
    # Only valid if "labels" is "inferred". This is the explicit list of class names (must match names of subdirectories).
    class_names=['left','right'],
    
    # One of "grayscale", "rgb", "rgba". Default: "rgb". Whether the images will be converted to have 1, 3, or 4 channels.
    color_mode="grayscale",
    
    # Size of the batches of data. Default: 32. If None, the data will not be batched.
    batch_size=batch_size,
    
    # Size to resize images to after they are read from disk, specified as (height, width). Defaults to (256, 256).
    image_size=(img_height, img_width),
    
    # Whether to shuffle the data. Default: True. If set to False, sorts the data in alphanumeric order.
    shuffle=True, 
    
    # Optional random seed for shuffling and transformations.
    seed=42,
    
    # Optional float between 0 and 1, fraction of data to reserve for validation.
    validation_split=0.2,
    
    # Subset of the data to return. One of "training", "validation" or "both". Only used if validation_split is set. When subset="both", the utility returns a tuple of two datasets
    subset='training'
)

validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    # Directory where the data is located. If labels is "inferred", it should contain subdirectories, each containing images for a class. 
    '../scratch/hand',
    
    # Either "inferred" (labels are generated from the directory structure), None (no labels), or a list/tuple of integer labels of the same size as the number of image files found in the directory. 
    labels='inferred',
    
    # String describing the encoding of labels. 'int': means that the labels are encoded as integers. (e.g. for sparse_categorical_crossentropy loss).
    # 'categorical' means that the labels are encoded as a categorical vector.  (e.g. for categorical_crossentropy loss).
    #'binary' means that the labels (there can be only 2) are encoded as float32 scalars with values 0 or 1. (e.g. for binary_crossentropy).
    label_mode = "categorical", 
    
    # Only valid if "labels" is "inferred". This is the explicit list of class names (must match names of subdirectories).
    class_names=['left','right'],
    
    # One of "grayscale", "rgb", "rgba". Default: "rgb". Whether the images will be converted to have 1, 3, or 4 channels.
    color_mode="grayscale",
    
    # Size of the batches of data. Default: 32. If None, the data will not be batched.
    batch_size=batch_size,
    
    # Size to resize images to after they are read from disk, specified as (height, width). Defaults to (256, 256).
    image_size=(img_height, img_width),
    
    # Whether to shuffle the data. Default: True. If set to False, sorts the data in alphanumeric order.
    shuffle=True, 
    
    # Optional random seed for shuffling and transformations.
    seed=42,
    
    # Optional float between 0 and 1, fraction of data to reserve for validation.
    validation_split=0.2,
    
    # Subset of the data to return. One of "training", "validation" or "both". Only used if validation_split is set. When subset="both", the utility returns a tuple of two datasets
    subset='validation'
)

# splits the validation_ds into validation and test data
test_ds = validation_ds.take(5)
validation_ds = validation_ds.skip(5)

# reserves 393 batches training
print('Batches for training -->', train_ds.cardinality())
# reserves 164 batches validation
print('Batches for validating -->', validation_ds.cardinality())
# reserves 5 batches testing
print('Batches for testing -->', test_ds.cardinality())

# Display the inferred class names from the dataset

You can find the class names in the class_names attribute on these datasets.

In [ ]:
# display the class names inferred from the training dataset

class_names = train_ds.class_names
print(class_names)

# Visualize the dataset images

Here are the first 9 images from the training dataset.

In [ ]:
# show the first 9 images in the training dataset

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"), cmap='gray')
    #TODO update labels
    #plt.title(class_names[labels[i]])
    plt.axis("off")

The image_batch is a tensor of the shape (32, 96, 96, 1). This is a batch of 32 images of shape 96x96x1 (the last dimension refers to color channels grayscaled). The label_batch is a tensor of the shape (32,), these are corresponding labels to the 32 images.

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

# Apply Data Augmentation

When you don't have a large image dataset or when your images are all set in a single direction like ours are, it's a good practice to artificially introduce sample diversity by applying random, yet realistic, transformations to the training images, such as rotation and horizontal flipping. This helps expose the model to different aspects of the training data and reduce overfitting. Learn more https://www.tensorflow.org/tutorials/images/data_augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  # randomly flips images during training
  tf.keras.layers.RandomFlip(
      # String indicating which flip mode to use. Can be "horizontal", "vertical", or "horizontal_and_vertical"
      'horizontal_and_vertical',
      
      # Integer. Used to create a random seed.
      seed=None
  ),
    
  # randomly rotates images during training
  tf.keras.layers.RandomRotation(
    # a float represented as fraction of 2 Pi, or a tuple of size 2 representing lower and upper bound for rotating clockwise and counter-clockwise. 
    # A positive values means rotating counter clock-wise, while a negative value means clock-wise. 
    0.2,
      
    # Points outside the boundaries of the input are filled according to the given mode (one of {"constant", "reflect", "wrap", "nearest"}).
    fill_mode='constant',
      
    # Supported values: "nearest", "bilinear".
    interpolation='nearest',
      
    # Integer. Used to create a random seed.
    seed=None,
      
    # the value to be filled outside the boundaries when fill_mode="constant".
    fill_value=0.0,
),
])

Visualize a few augmented examples by applying data augmentation to the same image several times:Visualize a few augmented examples by applying data augmentation to the same image several times:

In [ ]:
for image, _ in train_ds.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[4]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 1, cmap='gray')
    plt.axis('off')

# Configure the dataset for performance

Configure the dataset for performance. Let's make sure to use buffered prefetching so we can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data.

.cache() keeps the images in memory after they're loaded off disk during the first epoch
- caching a dataset, either in memory or on local storage. This will save some operations (like file opening and data reading) from being executed during each epoch.

.prefetch() overlaps data preprocessing and model execution while training.
- Prefetching overlaps the preprocessing and model execution of a training step. While the model is executing training step s, the input pipeline is reading the data for step s+1. Doing so reduces the step time to the maximum (as opposed to the sum) of the training and the time it takes to extract the data.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Define the training strategy, Hyperparameter Tune and Compile the model

For this tutorial, choose the tf.keras.optimizers.Adam optimizer and tf.keras.losses.SparseCategoricalCrossentropy loss function. To view training and validation accuracy for each training epoch, pass the metrics argument to Model.compile.

To do single-host, multi-device synchronous training with a Keras model, you would use the tf.distribute.MirroredStrategy API. Here's how it works:

- Instantiate a MirroredStrategy, optionally configuring which specific devices you want to use (by default the strategy will use all GPUs available).
- Use the strategy object to open a scope, and within this scope, create all the Keras objects you need that contain variables. Typically, that means creating & compiling the model inside the distribution scope.
- Train the model via fit() as usual.

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.



In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [ ]:
input_shape=(img_height, img_width, 1)

def model_builder(hp):
    model = keras.Sequential()
    data_augmentation
    # first CONV => RELU => POOL layer set
    model.add(Conv2D(
        # we define our first hyperparameter to search over — the number of filters in our CONV layer
        # The hyperparameter is given a name, conv_1, and can accept values in the range [32, 96] with steps of 32. 
        # This implies that valid values for conv_1 are 32, 64, 96
        data_format="channels_last", hp.Int("conv_1", min_value=16, max_value=64, step=32),
        (3, 3), padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # second CONV => RELU => POOL layer set
    model.add(Conv2D(
        # For our second CONV layer, we’re allowing more filters to be learned in the range [64, 128]. 
        # With a step size of 32, this implies that we’ll be testing values of 64, 96, 128
        hp.Int("conv_2", min_value=32, max_value=96, step=32),
        (3, 3), padding="same"))
    model.add(Activation("relu"))
    #model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # third CONV => RELU => POOL layer set
    model.add(Conv2D(
        # For our second CONV layer, we’re allowing more filters to be learned in the range [64, 128]. 
        # With a step size of 32, this implies that we’ll be testing values of 64, 96, 128
        hp.Int("conv_3", min_value=64, max_value=128, step=32),
        (3, 3), padding="same"))
    model.add(Activation("relu"))
    #model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # first (and only) set of FC => RELU layers
    #  We want to tune the number of nodes in this layer. We specify a minimum of 256 and a maximum of 768 nodes, allowing a step of 256
    model.add(Flatten())
    model.add(Dense(hp.Int("dense_units", min_value=128,
                           max_value=768, step=256)))
    model.add(Activation("relu"))
    #model.add(BatchNormalization())
    #model.add(Dropout(0.5))
    # softmax classifier
    #Apply a tf.keras.layers.Dense layer to convert these features into a single prediction per image.
    model.add(Dense(num_classes))
    # softmax results in this model performing around 50% accuracy, commented out
    #model.add(Activation("softmax"))
    
    # initialize the learning rate choices and optimizer
    # For our learning rate, we wish to see which of 1e-1, 1e-2, and 1e-3 performs best. 
    # Using hp.Choice will allow our hyperparameter tuner to select the best learning rate.
    lr = hp.Choice("learning_rate",
                   values=[1e-1, 1e-2, 1e-3])
    #opt = tf.keras.optimizers.Adam(learning_rate=lr)
    # compile the model
    model.compile(
        # optimizers are necessary for your model as they improve training speed and performance. Optional optimizers: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        # Loss function to calculate the models performance. The lower the loss, the closer our predictions are to the true labels.
        loss=tf.losses.BinaryCrossentropy(from_logits=True),
        # metrics to be evaluated by the model during training and testing.The strings 'accuracy' or 'acc', TF converts this to binary, categorical or sparse.
        metrics=['accuracy'],
          tf.keras.metrics.BinaryAccuracy(),
          tf.keras.metrics.FalseNegatives(),
        run_eagerly=None,
        # Int. Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead. 
        steps_per_execution=1,
    )
    # return the model
    return model

# Instantiate the tuner and perform hypertuning

The Keras Tuner has four tuners available:
1. RandomSearch
1. Hyperband
1. BayesianOptimization
1. Sklearn. 

In this tutorial, you use the Hyperband tuner. The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. 

To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).

In [ ]:
# Synchronous training across multiple replicas on one machine.
# a Keras model that was designed to run on a single-worker can seamlessly work on multiple workers with minimal code changes.
# https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras
strategy = tf.distribute.MirroredStrategy() # This strategy is typically used for training on one machine with multiple GPUs.
#strategy = tf.distribute.MultiWorkerMirroredStrategy(
#    cluster_resolver=None, 
#    communication_options=None) # A distribution strategy for synchronous training on multiple workers.
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# A model definition that doesn't take advantage of a training strategy

num_classes = len(class_names)
model_path = scratch_path + '/model'

# Open a strategy scope.
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    # nteger, the maximum number of epochs to train one model. It is recommended to set this to a value slightly higher than the expected epochs to convergence for your largest Model, and to use early stopping during trainin
    max_epochs=2,
    # Integer, the reduction factor for the number of epochs and number of models for each bracket. Defaults to 3.
    factor=3,
    # training strategy
    distribution_strategy=strategy,
    directory=scratch_path + '/model/model_hp',
    project_name='hypertune',
    #  If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. 
    # To disable this behavior, pass an additional overwrite=True argument while instantiating the tuner.
    overwrite=True
)

We’ll be using EarlyStopping to short circuit hyperparameter trials that are not performing well. Keep in mind that tuning hyperparameters is an extremely computationally expensive process, so if we can kill off poorly performing trials, we can save ourselves a bunch of time.

In [ ]:
# Stop training when a monitored metric has stopped improving.
stop_early = tf.keras.callbacks.EarlyStopping(
    # Quantity to be monitored.
    monitor='val_loss', 
    # Number of epochs with no improvement after which training will be stopped.
    patience=5,
    # training will stop when the quantity monitored has stopped decreasing "min" or increasing "max" or auto
    model="auto"
)

Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.

In [ ]:
tuner.search(train_ds, epochs=4, validation_data=validation_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print("[INFO] optimal number of filters in conv_1 layer: {}".format(
	best_hps.get("conv_1")))
print("[INFO] optimal number of filters in conv_2 layer: {}".format(
	best_hps.get("conv_2")))
print("[INFO] optimal number of filters in conv_2 layer: {}".format(
	best_hps.get("conv_3")))
print("[INFO] optimal number of units in dense layer: {}".format(
	best_hps.get("dense_units")))
print("[INFO] optimal learning rate: {:.4f}".format(
	best_hps.get("learning_rate")))

View all the layers of the network using the Keras Model.summary method:

# Train a model

Train the model for 10 epochs with the Keras Model.fit method:

In [ ]:
epochs = 10

# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    # x: Input data and y: Target data
    train_ds,
        
    # Number of samples per gradient update. If unspecified, batch_size will default to 32.
    batch_size=batch_size,
        
    # Data on which to evaluate the loss and any model metrics at the end of each epoch. The model will not be trained on this data. 
    validation_data=validation_ds,
        
    # Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
    epochs=epochs,
        
    # Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
    workers=1,
        
    # If True, use process-based threading. If unspecified, use_multiprocessing will default to False.  
    use_multiprocessing=False
)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
model.summary()

# Evaluate the model on test data

In [ ]:
model.evaluate(
    # x: Input data and y: Target data
    test_ds,
    
    # Integer or None. Number of samples per batch of computation.
    batch_size=batch_size,
    
    # "auto", 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = single line.
    verbose='auto',
    
    # Optional Numpy array of weights for the test samples, used for weighting the loss function. 
    sample_weight=None,
    
    # Integer or None. Total number of steps (batches of samples) before declaring the evaluation round finished. Ignored with the default value of None. 
    steps=None,
    
    # List of callbacks to apply during evaluation. See callbacks.
    callbacks=None,
    
    # Integer. Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
    max_queue_size=10,
    
    # Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
    workers=1,
    
    # If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
    use_multiprocessing=False
)

# Visualize the training results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Predict on new data

Use your model to classify an image that wasn't included in the training or validation sets.

In [ ]:
# un/comment test a left finger
path = scratch_path + '/real/10__M_Left_thumb_finger.png'

# un/comment test a right finger
#path = scratch_path + '/real/1__M_Right_ring_finger.png'

# Loads an image into PIL format.
img = tf.keras.utils.load_img(
    path,
    color_mode='grayscale',
    target_size=(img_height, img_width),
    interpolation='nearest',
    keep_aspect_ratio=False
)


plt.imshow(img, cmap='gray')

In [ ]:
# Converts a PIL Image instance to a Numpy array.
img_array = tf.keras.utils.img_to_array(img)

# Returns a tensor with a length 1 axis inserted at index axis.
img_array = tf.expand_dims(img_array, 0)

In [ ]:
# perform a prediction on the new fingerprint
predictions = model.predict(img_array)

In [ ]:
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

# Save the model

There are two formats you can use to save an entire model to disk: the TensorFlow SavedModel format, and the older Keras H5 format.

In [ ]:
# older Keras H5 format
model.save( scratch_path + '/model/hand_prediction.h5')

In [ ]:
# TensorFlow SavedModel format
model.save( scratch_path + '/model/hand_prediction.tf')

# Cleanup data

In [ ]:
!rm -rf ../scratch/*